# Homology of a Dowker complex

In this example we will
- generate a point cloud
- generate a collection of subsets (some would call this a *cover*, others would call it a *hypergraph*)
- compute the homology of the associated Dowker complex
- analyze cycle representatives
- plot cycle representatives

In [ ]:
import oat_python as oat

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import sklearn
import sklearn.metrics

## Generate a point cloud

In [ ]:
#   SET PARAMETER VALUES
npoints             =   25
maxdis              =   None
maxdim              =   1

#   GENERATE CLOUD FROM RANDOM SEED
cloud               =   oat.point_cloud.slice_of_sphere(npoints=npoints, random_seed=0)

#   PLOT
trace               =   go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers")
fig                 =   go.Figure([trace])
fig.update_layout( title=dict(text="Circle with noise"), width=800, height=800 )
fig.show()

## Choose a cover

We'll construct a cover by generating an epsilon net of the point cloud.  The cover will then consist of all radius-x neighbors of points in the net.

In [ ]:
###### THIS IS OLDER CODE; SEE BELOW FOR NEWER CODE; NEITHER GENERATES A DOWKER COMPLEX WITH THE HOMOLOGY SHOWN AT THE BOTTOM, HAVE TO GO BACK AND FIND THE PARAMETERS


dismat = sklearn.metrics.pairwise_distances(cloud)
dismat = (dismat + dismat.T)/2

#   COMPUTE A COVER

radius_neighbor = 1; # hyperedges will be neighborhoods of vertices, with this radius
radius_net = 0.5; # we'll make epsilon net with this value of epsilon

neighbors   = sklearn.neighbors.radius_neighbors_graph(cloud, radius=radius_neighbor, mode="connectivity", include_self=True)
neighbors = [ list(x.indices) for x in neighbors ]
for neighborhood in neighbors:
    neighborhood.sort()


perm, discurve      =   oat.dissimilarity.farthest_point( dismat, radius_net ) # compute an epsilon net
dowker_simplices    =   [ neighbors[x] for x in perm ]

#   PLOT THE COVER

trace = go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers")
fig = go.Figure(data=trace)

for v in perm:
    point = cloud[v]
    fig.add_shape(type="circle",
        xref="x", yref="y",
        fillcolor="PaleTurquoise",
        x0=point[0]-radius_neighbor, 
        y0=point[1]-radius_neighbor, 
        x1=point[0]+radius_neighbor, 
        y1=point[1]+radius_neighbor,
        line_color="LightSeaGreen",
        opacity=0.2,
    )    

fig.update_layout( title=dict(text="Hyperedges"), width=800, height=800 )
fig.show()

In [ ]:
#   COMPUTE A COVER

radius_neighbor     =   0.75
radius_net          =   0.5

# compute an epsilon net using farthest point sampling
net, _              =   oat.dissimilarity.farthest_point_with_cloud( 
                            cloud       =   cloud, 
                            epsilon     =   radius_net,
                        ) 

# cover elements / hyperedges will be neighborhoods of points in the net, with radius 1

neigh = sklearn.neighbors.NearestNeighbors()
neigh.fit(cloud)

dowker_simplices    =   [ sorted(list(x)) for x in neigh.radius_neighbors(cloud[net])[1] ]

Plot the hyperedges:

In [ ]:
trace = go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers")
fig = go.Figure(data=trace)

for v in net:
    point = cloud[v]
    fig.add_shape(type="circle",
        xref="x", yref="y",
        fillcolor="PaleTurquoise",
        x0=point[0]-radius_neighbor, 
        y0=point[1]-radius_neighbor, 
        x1=point[0]+radius_neighbor, 
        y1=point[1]+radius_neighbor,
        line_color="LightSeaGreen",
        opacity=0.2,
    )    

fig.update_layout( title=dict(text="Hyperedges"), width=800, height=800 )
fig.show()

## Compute homology

We'll compute the homology of
- the dual hypergraph; that is, the hypergraph where vertices are balls, and for each vertex `v` we have a hyperedge that contains every ball to which `v` belongs
- equivalently, the witness complex where every point is a witness and net points are landmarks

The homology solver only accepts hypergraphs represented by a list of lists of integers, currently.  If your hypergraph has a different format (e.g., if vertices are strings), then you can use some built-in tools to help translate back and forth between this format and list-of-list format; see the `rbs_reduced` notebook for examples.

In [ ]:
#   FACTOR THE BOUNDARY MATRIX

factored    =   oat.rust.FactoredBoundaryMatrixDowker( 
                    dowker_simplices            =   dowker_simplices, 
                    max_homology_dimension      =   2
                )

#   BETTI STATISTICS

print("\n\nBetti statistics\n")
display( factored.betti() )

#   HOMOLOGY

print("\nHomology\n")
homology    =   factored.homology()
display( homology )

#   CYCLE REPRESENTATIVE

print("\nCycle representative\n")
display( homology["cycle representative"][1] )


## Plot a cycle

In [ ]:
#   DATA FOR THE POINT CLOUD

data_cloud         = [ go.Scatter(x=cloud[:,0], y=cloud[:,1], mode="markers", name="Point cloud") ]

#   DATA FOR THE CYCLE

data_cycle = []
coo = cloud # coo stands for "coordinate oracle"
triangles = homology["cycle representative"][3]["simplex"].tolist() # list the triangles in the cycle
for count, triangle in enumerate(triangles):
    trace = oat.plot.triangle__trace2d( triangle, coo=coo ) # make the trace for each simplex
    trace.update(name=f"simplex {triangle}", text=f"simplex {triangle}", opacity=0.7) # customize
    data_cycle.append(trace) # append to the data group

#   PLOT

fig = go.Figure( data_cloud + data_cycle )
fig.update_layout(title="Toggle the legend entries to hide/reveal individual simplices!", height=700,width=800, template="plotly_white") 
fig.show()

# Group simplices into a single legend entry

Sometimes you don't want a separate legend entry for each simplex.  Here's an example that shows how to group them.

In [ ]:
triangles = homology["cycle representative"][3]["simplex"].tolist()
coo = cloud

#   point cloud
colors = [ np.arctan( v[1]/v[0] ) for v in cloud ]
# data_cloud         = [ go.Scatter(x=cloud[:,0], y=cloud[:,1], mode="markers", name="Point cloud", marker=dict(size= 7, color=colors, colorscale="Rainbow", line=dict(color="white", width=0.5) )) ]
data_cloud         = [ go.Scatter(x=cloud[:,0], y=cloud[:,1], mode="markers", name="Point cloud", marker=dict(size= 7, color=colors, colorscale="Rainbow", symbol="circle-open", line=dict( width=2) )) ]

#   the cycle
data_cycle = []
for count, triangle in enumerate(triangles):
    trace = oat.plot.triangle__trace2d( triangle, coo=coo ) # make the trace for each simplex
    trace.update( legendgroup="my_legend_group", name="cycle", text=f"simplex {triangle}", showlegend=(count==0), fillcolor="black", opacity=0.5) # customize
    data_cycle.append(trace) # append to the data group

fig = go.Figure( data_cloud + data_cycle )
fig.update_layout(title="Toggle all the simplices on and off, simultaneously, by clicking on `cycle`", height=700, width=750, template="plotly_dark") 
fig.show()

# Customize color patterns

and a custom sequence of symbols for [markers](https://plotly.com/python/marker-style/).

In [ ]:
colors = px.colors.qualitative.Bold
print(colors)

In [ ]:
triangles = homology["cycle representative"][3]["simplex"].tolist()
coo = cloud

#   point cloud
data_cloud         = [ go.Scatter( name="Point cloud", x=cloud[:,0], y=cloud[:,1], mode="markers+text", marker=dict(color="seagreen"), 
                        text=[str(p) for p in range(cloud.shape[0])],
                        textposition='top center', 
                        textfont=dict(color="seagreen", size=15)) ]

#   the cycle
data_cycle = []
for count, triangle in enumerate(triangles):
    color = colors[ count % len(colors) ] # choose another col
    trace = oat.plot.triangle__trace2d( triangle, coo=coo ) # make the trace for each simplex
    trace.update( name="cycle", legendgroup="1", showlegend=(count==0), text=f"simplex {triangle}", fillcolor=color, line=dict(color="white"), opacity=0.5) # customize
    data_cycle.append(trace) # append to the data group

fig = go.Figure( data_cloud + data_cycle )
fig.update_layout(title="Text color is sea green", height=700, width=750, template="ggplot2") 
fig.show()